In [256]:
import pandas as pd
import re
#from sklearn.metrics import mean_squared_error
#from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
pd.options.display.max_columns = 999
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [279]:
tmpX = pd.read_csv('train.csv')

In [280]:
def clean_cabin(x):
    try:
        return x[0]
    except TypeError:
        return "Missing"

In [281]:
#tmpX.Name = tmpX.Name.str.extract(r'(Mrs|Miss|Mr)')

In [282]:
tmpX.Ticket = tmpX.Ticket.str.extract(r'(\d{2,})')
tmpX.Ticket = tmpX.Ticket.fillna('1601')

In [283]:
tmpX['Cabin'] = tmpX.Cabin.apply(clean_cabin)

In [284]:
tmp1 = tmpX[tmpX.Pclass == 1]
tmp2 = tmpX[tmpX.Pclass == 2]
tmp3 = tmpX[tmpX.Pclass == 3]

In [285]:
tmp1.Age.fillna(tmp1.Age.median(),inplace=True)
tmp2.Age.fillna(tmp2.Age.median(),inplace=True)
tmp3.Age.fillna(tmp3.Age.median(),inplace=True)

In [286]:
tmp1['Cabin'].replace('Missing','C',inplace=True)
tmp2['Cabin'].replace('Missing','F',inplace=True)
tmp3['Cabin'].replace('Missing','F',inplace=True)

In [287]:
#tmp1.loc[(tmp1.Sex == 'male') & (tmp1.Name.isnull()),'Name'] = 'Mr'
#tmp1.loc[(tmp1.Sex == 'female') & (tmp1.Name.isnull()),'Name'] = 'Mrs'
#tmp2.loc[(tmp2.Sex == 'male') & (tmp2.Name.isnull()),'Name'] = 'Mr'
#tmp2.loc[(tmp2.Sex == 'female') & (tmp2.Name.isnull()),'Name'] = 'Mrs'
#tmp3.loc[(tmp3.Sex == 'male') & (tmp3.Name.isnull()),'Name'] = 'Mr'
#tmp3.loc[(tmp3.Sex == 'female') & (tmp3.Name.isnull()),'Name'] = 'Miss'

In [289]:
frames = [tmp1,tmp2,tmp3]
X = pd.concat(frames)

In [290]:
X['Fam'] = X.SibSp+tmpX.Parch
X.drop(['PassengerId','Name'],axis=1,inplace=True)

In [291]:
X.columns

Index([u'Survived', u'Pclass', u'Sex', u'Age', u'SibSp', u'Parch', u'Ticket',
       u'Fare', u'Cabin', u'Embarked', u'Fam'],
      dtype='object')

In [292]:
X.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Cabin       0
Embarked    2
Fam         0
dtype: int64

In [293]:
categorical_variables = ['Sex', 'Embarked','Pclass','Cabin','SibSp','Parch']

for variable in categorical_variables:
    X[variable].fillna("Missing", inplace=True)
    dummies = pd.get_dummies(X[variable], prefix=variable)
    X = pd.concat([X, dummies], axis=1)
    X.drop([variable], axis=1, inplace=True)

In [294]:
scaler = StandardScaler()
y = X['Survived']
y_log = X['Survived']
X.drop(['Survived'],axis=1,inplace=True)
X_log = scaler.fit_transform(X)

In [295]:
X_train, X_test, y_train, y_test = train_test_split(X_log,y_log, test_size=.2, random_state=42)

In [331]:
model_log = LogisticRegression(penalty='l1', C=1.0)
model_log.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [332]:
accuracy_score(y_test,model_log.predict(X_test))

0.88268156424581001

In [333]:
#logit_roc_auc = 
roc_auc_score(y_test, model_log.predict(X_test))

0.87623891497130935

In [299]:
model = RandomForestRegressor(n_estimators=200, 
                              oob_score=True, 
                              n_jobs=-1, 
                              random_state=42, 
                              max_features=0.9, 
                              min_samples_leaf=3)
model.fit(X, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=0.9, max_leaf_nodes=None, min_samples_leaf=3,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=-1, oob_score=True, random_state=42,
           verbose=0, warm_start=False)

In [300]:
roc = roc_auc_score(y, model.oob_prediction_)
print "C-stat: ", roc

C-stat:  0.882439097136


In [301]:
log_mod = model_log.predict(X_log)
rf_mod = model.oob_prediction_

In [311]:
combo = (log_mod+rf_mod)/2

# FINAL SCORE

In [312]:
roc_auc_score(y, combo)

0.88460678106924884

# Forest Testing

In [215]:
results = []
n_estimator_options = [50, 100, 200, 500, 1000]
max_features_options = ["auto", "sqrt", "log2", 0.9, 0.2]
min_samples_leaf_options = [2, 3, 4, 5, 6]

for trees in n_estimator_options:
    for features in max_features_options:
        for min_samples in min_samples_leaf_options:
            model = RandomForestRegressor(n_estimators=trees, 
                                          oob_score=True, 
                                          n_jobs=-1, 
                                          random_state=42, 
                                          max_features=features, 
                                          min_samples_leaf=min_samples)
            model.fit(X, y)
            print trees, "trees", features, "features", min_samples, "min samples"
            roc = roc_auc_score(y, model.oob_prediction_)
            print "C-stat: ", roc
            results.append(roc)
            print ""
    
#pd.Series(results, min_samples_leaf_options).plot();

50 trees auto features 2 min samples
C-stat:  0.866900478275

50 trees auto features 3 min samples
C-stat:  0.872548706313

50 trees auto features 4 min samples
C-stat:  0.873299672983

50 trees auto features 5 min samples
C-stat:  0.8745512841

50 trees auto features 6 min samples
C-stat:  0.87758976981

50 trees sqrt features 2 min samples
C-stat:  0.876149085525

50 trees sqrt features 3 min samples
C-stat:  0.87316119686

50 trees sqrt features 4 min samples
C-stat:  0.874311613886

50 trees sqrt features 5 min samples
C-stat:  0.876596469924

50 trees sqrt features 6 min samples
C-stat:  0.870641996613

50 trees log2 features 2 min samples
C-stat:  0.876149085525

50 trees log2 features 3 min samples
C-stat:  0.87316119686

50 trees log2 features 4 min samples
C-stat:  0.874311613886

50 trees log2 features 5 min samples
C-stat:  0.876596469924

50 trees log2 features 6 min samples
C-stat:  0.870641996613

50 trees 0.9 features 2 min samples
C-stat:  0.871422256309

50 trees 0.9 f